In [ ]:
from google.colab import files
files.upload()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
plt.style.use('ggplot') #using a built-in type

In [ ]:
df = pd.read_excel("Reviews.xlsx")
df.head()



In [ ]:
df['Score'].value_counts().sort_index().plot(kind='bar', title='number of star reviews')
plt.show()

Some of the ntlk lib functions !

In [ ]:
example = df['Text'][100]
print(example)
nltk.download('punkt')
nltk.download('punkt_tab')
tokens = nltk.word_tokenize(example)
tokens[:10]


nltk.download('averaged_perceptron_tagger_eng')
tagged = nltk.pos_tag(tokens)

nltk.download('maxent_ne_chunker_tab')
nltk.download('words')
entities = nltk.chunk.ne_chunk(tagged) #chunking the tokens into grps of the type (determiner + adj (if any) + noun)
entities.pprint()

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer #VADER sentiment analyzer
from tqdm.notebook import tqdm #progress bar


nltk.download('vader_lexicon')
senti = SentimentIntensityAnalyzer()
print(senti.polarity_scores('i really like this hello kitty plushie!'))
print(example)
print(senti.polarity_scores(example))

result = {}
for i, row in tqdm(df.iterrows(), total = len(df)):
  text = row['Text']
  myid = row['Id']
  result[myid]= senti.polarity_scores(text)



In [ ]:
#to join df and senti data together
vaders = pd.DataFrame(result).T #T transposes the dict
vaders = vaders.reset_index().rename(columns={'index':'Id'})
vaders = vaders.merge(df,how='inner')


In [ ]:
#visualizing results !! (assuming a higher score means the product is evoking positive emotions and vice versa)
graph =sns.barplot(data=vaders,x='Score',y='compound',palette='pastel')
graph.set_title("Score given by Amazon review vs comp score by senti")
graph.set_ylabel("compound score")
graph.set_ylim(-0.5, 1)

The Roberta Pretrained Model !

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification #used for text sentiment analysis
from scipy.special import softmax #Taking a vector and converting it into a prob dist

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment" #model is fine tuned on twitter data !
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict




In [ ]:

res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        text = row['Text']
        myid = row['Id']
        vader_result = senti.polarity_scores(text)
        vader_result_rename = {}
        for key, value in vader_result.items():
            vader_result_rename[f"vader_{key}"] = value
        roberta_result = polarity_scores_roberta(text)
        both = {**vader_result_rename, **roberta_result}
        res[myid] = both
    except RuntimeError:
        print(f'Broke for id {myid}')

In [ ]:
results_df = pd.DataFrame(res).T
results_df = results_df.reset_index().rename(columns={'index': 'Id'})
results_df = results_df.merge(df, how='left')
results_df.head()


Comparing Results of the VADER model with the ROBERTA model

In [ ]:
results_df.columns

In [ ]:
sns.scatterplot(data=results_df, x='Score', y='roberta_pos')
plt.title('Amazon Review Score vs. RoBERTa Positive Sentiment Score')
plt.xlabel('Amazon Score')
plt.ylabel('RoBERTa Positive Sentiment Score')
plt.show()

In [ ]:
sns.scatterplot(data=results_df, x='Score', y='vader_compound')
plt.title('Amazon Review Score vs. VADER Compound Sentiment Score')
plt.xlabel('Amazon Score')
plt.ylabel('VADER Compound Sentiment Score')
plt.show()

Much less outliers in the Roberta Model!